# Step 3
- Determine the topics in the conversations of chosen characters

In [2]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
# nltk.download('averaged_perceptron_tagger')

In [3]:
sheets = ['episode1', 'episode2', 'episode3', 'episode4', 'episode5', 'episode6']
data = pd.concat([pd.read_excel('data.xlsx', sheet_name = sheet) for sheet in sheets])

In [4]:
df = (data
     .assign(dialogue=lambda df_: df_.dialogue.str.replace('\s+', ' ', regex=True).str.strip())
     .assign(sorted_pairs=lambda df_: df_.apply(lambda df_: '-'.join(sorted([df_['initiator'], df_['responder']])), axis=1))
     .assign(weight=lambda df_: df_.groupby(['sorted_pairs'])['sorted_pairs'].transform('count'))
     .assign(dialogue=lambda df_: df_.groupby(['sorted_pairs'])['dialogue'].transform(lambda df_: ' '.join(df_.unique())))
     .drop_duplicates('sorted_pairs')
     .drop(columns='sorted_pairs')
    )

In [5]:
import gensim
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
# nltk.download('averaged_perceptron_tagger')

# func to add topics
def add_topic_labels(df):
    # create a list of documents from the 'script' column
    documents = list(df['dialogue'])

    # define stop words, tags to remove, and words to remove
    stop_words = stopwords.words('english')
    removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
    remove_words = ['like','gone','know','right','na','gon','yeah','really','okay','even','sure','miller','lisa',
                    'get','gonna','well','thank','oh','uh','hi','got','um','go','little','mortimer','mayfield','though',
                    'would','great','come','hey','wanna','hmm','mr','yes','good','give','jorgy','coleman','ruth','also',
                    'going','want','let','think','one','us','look','make','ed','gotta','ben','dale','randolph','actually',
                    'something','back','see','need','man','say','sorry','could','madoff','wrong','eric','robert','saw',
                    'thing','lt','lf','made','way','said','sir','two','new','naomi','ha','ok','around','jean','ron','mm','onto',
                    'louis','winthorpe','gekko','gordon','buddy','jordan','tell','erin','sam','eric','seen','michael','seth',
                    'patrick','paul','allen','listen','god','bateman','marcus','heh','bernie','big','steve','jimmy','mrs',
                    'anything','christie','patrick','dorsia','garfield','mark','maybe','nick','donnie','name','went'
                   ]

    # remove stop words and unwanted words
    tokenized_docs = [[token for token in gensim.utils.simple_preprocess(doc) if token not in stop_words and token not in remove_words] for doc in documents]

    # remove tags
    tagged_docs = [nltk.pos_tag(doc) for doc in tokenized_docs]
    tokenized_docs = [[token for token, pos in doc if pos not in removal] for doc in tagged_docs]

    # create a dictionary from the tokenized documents
    dictionary = corpora.Dictionary(tokenized_docs)

    # create a bag-of-words representation of the documents
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

    # train an LDA model on the corpus
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=5,
                                                random_state=42,
                                                passes=10)

    # extract the top fifteen words for each topic
    topic_labels = []
    for doc in tokenized_docs:
        bow = dictionary.doc2bow(doc)
        topics = lda_model.get_document_topics(bow)
        top_fifteen_words = [dictionary[word] for word, prob in sorted(lda_model.get_topic_terms(max(topics, key=lambda x: x[1])[0], topn=15), key=lambda x: x[1], reverse=True)]
        topic_labels.append(top_fifteen_words)

    # add the topic labels as a new column in the dataframe
    df['topic'] = topic_labels
    return df

In [6]:
# Define the names to create DataFrames for
chosen_names = ['NICOLE', 'TANYA', 'RACHEL', 'SHANE', 'ARMOND', 'PAULA', 'OLIVIA']

# Loop through the chosen names and create a new DataFrame for each one
for name in chosen_names:
    # Select rows where the name appears in either initiator or responder
    new_df = df[(df['initiator'] == name) | (df['responder'] == name)]
    
    # Create a new DataFrame with the selected rows
    globals()[f'{name}_df'] = pd.DataFrame(new_df)
    
# func to add word_count column
def tweak_df(df):
    return (df
             .assign(word_count=lambda df_: df_.dialogue.str.len().astype('int16'))
             .astype({'weight': 'int8'})
             .sort_values('word_count', ascending=False)
             .reset_index()
             .drop(columns='index')
            )

# list of dataframes to tweak
tweak_these = [NICOLE_df, TANYA_df, RACHEL_df, SHANE_df, ARMOND_df, PAULA_df, OLIVIA_df]

for i in range(len(tweak_these)):
    tweak_these[i] = tweak_df(tweak_these[i])

NICOLE_df, TANYA_df, RACHEL_df, SHANE_df, ARMOND_df, PAULA_df, OLIVIA_df = tweak_these

In [7]:
## NICOLE
# create mapping to create dataframe for each movie
characters = {'NICOLE_OLIVIA':0, 'NICOLE_MARK':1, 'RACHEL_NICOLE':2}

# loop through dictionary to create dataframes
for key, value in characters.items():
    NICOLE_df_key = pd.DataFrame([[NICOLE_df.loc[value, 'initiator'], NICOLE_df.loc[value, 'responder'],
                                   NICOLE_df.loc[value, 'dialogue']]],
                                 columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = NICOLE_df_key

In [ ]:
## TANYA
# create mapping to create dataframe for each movie
characters = {'TANYA_BELINDA':0, 'GREG_TANYA':1, 'TANYA_SHANE':2}

# loop through dictionary to create dataframes
for key, value in characters.items():
    TANYA_df_key = pd.DataFrame([[TANYA_df.loc[value, 'initiator'], TANYA_df.loc[value, 'responder'],
                                  TANYA_df.loc[value, 'dialogue']]],
                                columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = TANYA_df_key

In [ ]:
## RACHEL
# create mapping to create dataframe for each movie
characters = {'RACHEL_SHANE':0, 'KITTY_RACHEL':1, 'BELINDA_RACHEL':6}

# loop through dictionary to create dataframes
for key, value in characters.items():
    RACHEL_df_key = pd.DataFrame([[RACHEL_df.loc[value, 'initiator'], RACHEL_df.loc[value, 'responder'],
                                   RACHEL_df.loc[value, 'dialogue']]],
                                 columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = RACHEL_df_key

In [ ]:
## SHANE
# create mapping to create dataframe for each movie
characters = {'ARMOND_SHANE':1, 'SHANE_KITTY':2, 'SHANE_OLIVIA':5}

# loop through dictionary to create dataframes
for key, value in characters.items():
    SHANE_df_key = pd.DataFrame([[SHANE_df.loc[value, 'initiator'], SHANE_df.loc[value, 'responder'],
                                  SHANE_df.loc[value, 'dialogue']]], columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = SHANE_df_key

In [ ]:
## ARMOND
# create mapping to create dataframe for each movie
characters = {'ARMOND_MARK':1, 'ARMOND_DILLON':2, 'ARMOND_LANI':3}

# loop through dictionary to create dataframes
for key, value in characters.items():
    ARMOND_df_key = pd.DataFrame([[ARMOND_df.loc[value, 'initiator'], ARMOND_df.loc[value, 'responder'], ARMOND_df.loc[value, 'dialogue']]], columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = ARMOND_df_key

In [ ]:
## PAULA
# create mapping to create dataframe for each movie
characters = {'OLIVIA_PAULA':0, 'PAULA_KAI':1, 'MARK_PAULA':4}

# loop through dictionary to create dataframes
for key, value in characters.items():
    PAULA_df_key = pd.DataFrame([[PAULA_df.loc[value, 'initiator'], PAULA_df.loc[value, 'responder'], PAULA_df.loc[value, 'dialogue']]], columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = PAULA_df_key

In [ ]:
## OLIVIA
# create mapping to create dataframe for each movie
characters = {'MARK_OLIVIA':2, 'TANYA_OLIVIA':3, 'OLIVIA_QUINN':6}

# loop through dictionary to create dataframes
for key, value in characters.items():
    OLIVIA_df_key = pd.DataFrame([[OLIVIA_df.loc[value, 'initiator'], OLIVIA_df.loc[value, 'responder'], OLIVIA_df.loc[value, 'dialogue']]], columns=['initiator', 'responder', 'dialogue']).pipe(add_topic_labels)
    globals()[key] = OLIVIA_df_key

In [ ]:
# List of dataframes
dataframes = [MARK_OLIVIA, TANYA_OLIVIA, OLIVIA_QUINN, OLIVIA_PAULA, PAULA_KAI, MARK_PAULA,
              ARMOND_MARK, ARMOND_DILLON, ARMOND_LANI, ARMOND_SHANE, SHANE_KITTY, SHANE_OLIVIA,
              RACHEL_SHANE, KITTY_RACHEL, BELINDA_RACHEL, TANYA_BELINDA, GREG_TANYA, TANYA_SHANE,
              NICOLE_OLIVIA, NICOLE_MARK, RACHEL_NICOLE
             ]


# Initialize an empty dataframe to store the concatenated data
conv_df = pd.DataFrame()

# Concatenate the dataframes vertically
for df in dataframes:
    conv_df = pd.concat([conv_df, df], axis=0)

# Reset the index of the concatenated dataframe
conv_df = conv_df.reset_index(drop=True)

In [ ]:
# conv_df.loc[0, ['topic']].tolist()

## Topic conversations of interesting characters

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams


# SET VIZ TEMPLATE
# colors
YELLOW='yellow'
BLACK='#1a1a1a'
GREY='#696969'
WHITE='white'

# fonts
futura_font_path = 'graphfonts/Futura/futura-medium-condensed.ttf'
paypal_font_path = 'graphfonts/Paypal/PayPalSansSmall-Regular.ttf'
FUTURA = fm.FontProperties(fname=futura_font_path)
PAYPAL = fm.FontProperties(fname=paypal_font_path)

DPI=300

#func to plot topics
def plot_word_grid(words, suptitle):
    # Define the figure size and background color
    fig = plt.figure(figsize=(8, 4), facecolor=BLACK, dpi=DPI)

    # Set the title
    fig.suptitle(suptitle, fontproperties=FUTURA, color='white', fontsize=12, y=.93)

    # Define the layout of the subplots
    layout = (5, 5)

    # Create subplots and add words to them
    for i, word in enumerate(words):
        ax = fig.add_subplot(*layout, i+1)
        ax.set_facecolor(BLACK)
        ax.text(0.5, 0.5, word, fontsize=10, ha='center', va='center', fontproperties=PAYPAL, color=YELLOW)

        # Remove spines and tick labels
        for side in 'top,right,bottom,left'.split(','):
            ax.spines[side].set_visible(False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_xticklabels([])
        ax.set_yticklabels([])

    # Adjust the spacing between subplots
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
    
    # Show the figure
    plt.show()

In [ ]:
conv_df

In [ ]:
plot_word_grid(conv_df.loc[9, 'topic'], 'ARMOND and SHANE conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[10, 'topic'], 'SHANE and KITTY conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[19, 'topic'], 'NICOLE and MARK conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[20, 'topic'], 'RACHEL and NICOLE conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[4, 'topic'], 'PAULA and KAI conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[15, 'topic'], 'TANYA and BELINDA conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[13, 'topic'], 'KITTY and RACHEL conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[16, 'topic'], 'GREG and TANYA conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[3, 'topic'], 'OLIVIA and PAULA conversation topics')

In [ ]:
plot_word_grid(conv_df.loc[2, 'topic'], 'OLIVIA and QUINN conversation topics')